In [1]:
import header
from header import __root__
from src import gs

🔑 Found password in password.txt (DEBUG MODE)
✅ Successfully opened KeePass database: C:\Users\user\Documents\repos\hypotez\secrets\credentials.kdbx
Failed to load GAPI credentials


2025-05-16 06:41:08,512 - DEBUG - 🐛 Нет нового релиза: https://github.com/repos/hypotez/hypo/releases/latest
 response.status_code=404 


In [2]:
import pandas as pd
import zipfile
import time
from pathlib import Path
from typing import Optional, List, Dict, Any
from types import SimpleNamespace
from src.utils.jjson import j_loads, j_loads_ns
from src.utils.file import read_text_file, recursively_yield_file_path
from src.utils.printer import pprint as print
from src.logger import logger

In [3]:
class Config:
    """Класс конфигурации скрипта."""
    ENDPOINT: Path = __root__ / 'SANDBOX' / 'davidka'
    config: SimpleNamespace = j_loads_ns(ENDPOINT / 'davidka.json')
    STORAGE: Path
    try:
        STORAGE: Path = Path(config.local_storage.storage) if config.actual_storage == 'local_storage' else Path(config.google_drive.storage)
    except Exception as ex:
        raise ValueError(f"Ошибка загрузки пути к хранилищу: ", ex)
        
    TRAIN_STORAGE:Path = STORAGE / 'train_data' # / 'product'
    UZZIPPED_STORAGE:Path = STORAGE / 'unzipped_storage' / 'product'
    GEMINI_API_KEY: Optional[str] = gs.credentials.gemini.onela.api_key # Установить имя владельца ключа (onela, kazazrinov, ...)
    GEMINI_MODEL_NAME: str = 'gemini-2.0-flash-exp' # Актуальное имя модели может отличаться, например, 'gemini-1.5-flash-latest'
    
    

In [27]:
train_data: list = []
counter: int = 0

for file in recursively_yield_file_path(Config.TRAIN_STORAGE, '*.json'):
    try:
        train_data.extend( j_loads(file) )
        counter += 1
        #print(f"{counter}: {file} — {len(products_list_from_file)} items")
    except Exception as ex:
        logger.error(f"Ошибка при чтении {file}", ex)

In [28]:
#print(train_data[:5])

In [29]:
df = pd.DataFrame(train_data)

In [30]:
print(len(df))

89425


In [31]:
df.head(2)

,text,label
0,Virtual Worlds for Real Life\nCrafted to Cruis...,1
1,HD Hyundai Xitesolution\nHD Hyundai Xitesoluti...,22


In [32]:
df.columns

Index(['text', 'label'], dtype='object')

In [ ]:
#print(len(df.columns))

In [ ]:
#df.head

In [ ]:
#df_cleaned = df_transposed.dropna(subset=['supplier'])

In [ ]:
#supplier_list = df_cleaned['supplier'].dropna().unique()
#logger.info(supplier_list)

In [ ]:
#supplier_counts = df_clean.groupby('supplier').count().reset_index()

In [ ]:
#print(supplier_counts)

In [ ]:
#df_transposed.columns

## В следующей функции возникает ошибка скорее всего из-за неправильного символа UNICODE в названии столбца

In [ ]:
"""
columns_to_check = ['category']
actual_category_column_name = None
for c_name in df_transposed.columns:
    if 'category' in c_name: # или c_name == 'category'
        actual_category_column_name = c_name
        break

if actual_category_column_name:
    columns_to_check = [actual_category_column_name]
else:
    columns_to_check = [] # 'category' не найдена

existing_columns = [col for col in columns_to_check if col in df_transposed.columns]


# Если колонки существуют, считаем товары
if existing_columns:
    # Объединяем значения из существующих колонок
    category_counts = pd.concat([df[col] for col in existing_columns])

    # Считаем количество товаров в каждой категории
    category_counts = category_counts.dropna().value_counts().reset_index()
    category_counts.columns = ['category', 'product_count']
else:
    category_counts = pd.DataFrame(columns=['category', 'product_count'])

print(category_counts)
"""

✅ 1. Частота значений (кол-во на каждую категорию):

In [ ]:
df['category'].value_counts()

In [ ]:
df_transposed.to_csv(Config.TRAIN_STORAGE / 'train.csv', index=False, encoding='utf-8')